In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# divece congif
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'selected device is: {device}')

selected device is: cpu


In [3]:
LR = 0.01
EPOCHES = 5
BATCH_SIZE = 64

In [4]:
# MINST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5))
])

In [5]:
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transform)

* 70,000 grayscale images of handwritten digits:

* 60,000 for training

* 10,000 for testing

* Each image is 28x28 pixels

* Digits: 0 to 9

* Labelled dataset (each image has the correct digit as a label)

In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
# the CNN

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1) # output: 16x28x28
        self.pool = nn.MaxPool2d(2,2) # # output: 16x14x14
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1) # output: 32x14x14 -> pool -> 32x7x7
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10) # 10 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        

In [8]:
model = SimpleCNN().to(device)

In [9]:
critetion = nn.CrossEntropyLoss() # no need to use softmax 
optimizer = optim.Adam(model.parameters(), lr=LR)

In [11]:
# training loop
def train(model, train_loader, critetion, optimizer, EPOCHES):
    model.train()
    
    for epoch in range(EPOCHES):
        running_loss = 0.0
        loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{EPOCHES}]")
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = critetion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            loop.set_postfix(loss=loss.item())
        print(f"Epoch [{epoch+1}/{EPOCHES}], Loss: {loss.item():.4f}")

        

In [12]:
# train the model
train(model, train_loader, critetion, optimizer, EPOCHES)

Epoch [1/5]:   0%|          | 0/938 [01:15<?, ?it/s, loss=0.0373] 

Epoch [1/5], Loss: 0.0373


Epoch [1/5]:   0%|          | 0/938 [01:15<?, ?it/s, loss=0.0373]


Epoch [2/5], Loss: 0.2106


Epoch [3/5]:   0%|          | 0/938 [01:04<?, ?it/s, loss=0.0532] 

Epoch [3/5], Loss: 0.0532


Epoch [3/5]:   0%|          | 0/938 [01:04<?, ?it/s, loss=0.0532]


Epoch [4/5], Loss: 0.0195


Epoch [5/5]:   0%|          | 0/938 [01:19<?, ?it/s, loss=0.0168]  


Epoch [5/5], Loss: 0.0168


In [15]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 97.47%


In [20]:
save_file_path = "./models/"
torch.save(model.state_dict(), f"{save_file_path}Simple_CNN_model.pth")
print("Model saved successfully!")

Model saved successfully!
